In [185]:
import os

In [186]:
%pwd

'D:\\ML PROJECTS\\Wine-Quality-Prediction-with-MLOps'

In [187]:
os.chdir('D:\ML PROJECTS\Wine-Quality-Prediction-with-MLOps')

In [188]:
%pwd

'D:\\ML PROJECTS\\Wine-Quality-Prediction-with-MLOps'

In [189]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [190]:
from WineQPrediction.constants import *
from WineQPrediction.utils.common import read_yaml, create_directories

In [191]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [192]:
import os
import urllib.request as request
import zipfile
from WineQPrediction import logger
from WineQPrediction.utils.common import get_size

In [193]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
  

In [194]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-06-01 18:08:48,826]: INFO: common yaml file: config\config.yaml loaded successfully]
[2024-06-01 18:08:48,828]: INFO: common yaml file: params.yaml loaded successfully]
[2024-06-01 18:08:48,831]: INFO: common yaml file: Schema.yaml loaded successfully]
[2024-06-01 18:08:48,834]: INFO: common created directory at: artifacts_root]
[2024-06-01 18:08:48,836]: INFO: common created directory at: artifacts/data_ingestion]
[2024-06-01 18:08:50,886]: INFO: 2366278714 artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 0FEE:384493:17EBAF:2032A0:665B1659
Accept-Ranges: bytes
Date: Sa